In [1]:
#default_exp data.importer.buoy_data

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
#export
from huey.imports import *

[2020-07-06 09:18:19,216] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=20403


# Import Buoy Realtime Wave Data

In [ ]:
station_id = "46025"

In [4]:
station_id = "46053"

In [56]:
station_id = "46011"

## In Steps

In [57]:
realtime_url = f"https://www.ndbc.noaa.gov/data/realtime2/{station_id}.spec"

In [58]:
realtime_url

'https://www.ndbc.noaa.gov/data/realtime2/46011.spec'

In [59]:
df = pd.read_csv(realtime_url, delim_whitespace=True)

In [24]:
df.head()

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
0,#yr,mo,dy,hr,mn,m,m,sec,m,sec,-,degT,-,sec,degT
1,2020,07,06,15,40,2.9,MM,MM,MM,MM,MM,MM,STEEP,6.4,316
2,2020,07,06,14,40,2.7,MM,MM,MM,MM,MM,MM,AVERAGE,6.2,316
3,2020,07,06,13,40,2.8,MM,MM,MM,MM,MM,MM,AVERAGE,6.4,322
4,2020,07,06,12,40,3.2,MM,MM,MM,MM,MM,MM,STEEP,6.7,316


In [60]:
len(df)

1095

In [ ]:
df = df.replace('MM', np.NaN)

In [ ]:
df.tail()

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
1073,2020,05,20,04,40,1.7,1.6,5.9,0.6,3.0,W,W,VERY_STEEP,5.4,265
1074,2020,05,20,03,40,1.9,1.3,16.0,1.4,5.9,S,W,VERY_STEEP,5.7,267
1075,2020,05,20,02,40,1.9,1.2,6.7,1.4,5.9,W,W,VERY_STEEP,5.6,267
1076,2020,05,20,01,40,1.9,1.0,14.8,1.6,5.9,SSW,W,VERY_STEEP,5.7,268
1077,2020,05,20,00,40,1.8,1.0,13.8,1.6,6.2,SSW,W,VERY_STEEP,5.6,268


In [ ]:
sql = f"select * from buoy_realtime_wave_detail order by ts desc limit 1"

In [ ]:
sql

'select * from buoy_realtime_wave_detail order by ts desc limit 1'

In [ ]:
from huey.data.sql import DB

In [ ]:
db = DB()
db_engine = db.engine

[2020-07-03 17:40:14,497] {base_hook.py:84} INFO - Using connection to: id: huey_dev. Host: localhost, Port: None, Schema: huey_dev, Login: postgres, Password: XXXXXXXX, extra: None


In [ ]:
result = pd.read_sql(sql, db_engine)

In [ ]:
result

,id,buoy_id,ts,significant_wave_height,swell_height,swell_period,swell_direction,wind_wave_height,wind_wave_period,steepness,average_wave_period,dominant_wave_direction,created_at,updated_at,wind_wave_direction
0,9320,436,2020-07-03 22:40:00+00:00,0.8,0.7,19.0,SW,0.3,4.3,SWELL,None,227,None,None,None


In [ ]:
df.head()

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
0,#yr,mo,dy,hr,mn,m,m,sec,m,sec,-,degT,-,sec,degT
1,2020,07,03,23,40,1.0,0.9,19.0,0.4,4.8,S,W,SWELL,7.3,188
2,2020,07,03,22,40,0.8,0.7,19.0,0.3,4.3,SW,WSW,SWELL,7.1,227
3,2020,07,03,21,40,0.7,0.7,19.0,0.2,4.2,SW,WSW,NaN,6.9,232
4,2020,07,03,20,40,0.7,0.6,13.8,0.2,2.2,SW,W,NaN,6.6,219


In [ ]:
len(df)

1078

In [ ]:
df2 = df.rename(columns={
    "WVHT": "significant_wave_height",
    "SwH": "swell_height",
    "SwP": "swell_period",
    "SwD": "swell_direction",
    "WWH": "wind_wave_height",
    "WWP": "wind_wave_period",
    "WWD": "wind_wave_direction",
    "STEEPNESS": "steepness",
    "APD": "average_wave_period",
    "MWD": "dominant_wave_direction"
})

In [ ]:
df2.head()

,#YY,MM,DD,hh,mm,significant_wave_height,swell_height,swell_period,wind_wave_height,wind_wave_period,swell_direction,wind_wave_direction,steepness,average_wave_period,dominant_wave_direction
0,#yr,mo,dy,hr,mn,m,m,sec,m,sec,-,degT,-,sec,degT
1,2020,07,03,23,40,1.0,0.9,19.0,0.4,4.8,S,W,SWELL,7.3,188
2,2020,07,03,22,40,0.8,0.7,19.0,0.3,4.3,SW,WSW,SWELL,7.1,227
3,2020,07,03,21,40,0.7,0.7,19.0,0.2,4.2,SW,WSW,NaN,6.9,232
4,2020,07,03,20,40,0.7,0.6,13.8,0.2,2.2,SW,W,NaN,6.6,219


In [ ]:
df2 = df2[1:]

In [ ]:
df2.head()

,#YY,MM,DD,hh,mm,significant_wave_height,swell_height,swell_period,wind_wave_height,wind_wave_period,swell_direction,wind_wave_direction,steepness,average_wave_period,dominant_wave_direction
1,2020,07,03,23,40,1.0,0.9,19.0,0.4,4.8,S,W,SWELL,7.3,188
2,2020,07,03,22,40,0.8,0.7,19.0,0.3,4.3,SW,WSW,SWELL,7.1,227
3,2020,07,03,21,40,0.7,0.7,19.0,0.2,4.2,SW,WSW,NaN,6.9,232
4,2020,07,03,20,40,0.7,0.6,13.8,0.2,2.2,SW,W,NaN,6.6,219
5,2020,07,03,19,40,0.8,0.7,19.0,0.2,4.3,WSW,W,NaN,8.2,237


In [ ]:
row = df2.iloc[0];
row

#YY                         2020
MM                            07
DD                            03
hh                            23
mm                            40
significant_wave_height      1.0
swell_height                 0.9
swell_period                19.0
wind_wave_height             0.4
wind_wave_period             4.8
swell_direction                S
wind_wave_direction            W
steepness                  SWELL
average_wave_period          7.3
dominant_wave_direction      188
Name: 1, dtype: object

In [ ]:
def ts_from_df_row(row):
        return datetime(
            int(row['#YY']), 
            int(row['MM']), 
            int(row['DD']), 
            int(row['hh']), 
            int(row['mm']), 
            tzinfo=timezone.utc)

In [ ]:
ts_from_df_row(row)

datetime.datetime(2020, 7, 3, 23, 40, tzinfo=datetime.timezone.utc)

In [ ]:
df2["ts"] = df2.apply(lambda row: ts_from_df_row(row), axis=1)

In [ ]:
df2.head()

,#YY,MM,DD,hh,mm,significant_wave_height,swell_height,swell_period,wind_wave_height,wind_wave_period,swell_direction,wind_wave_direction,steepness,average_wave_period,dominant_wave_direction,ts
1,2020,07,03,23,40,1.0,0.9,19.0,0.4,4.8,S,W,SWELL,7.3,188,2020-07-03 23:40:00+00:00
2,2020,07,03,22,40,0.8,0.7,19.0,0.3,4.3,SW,WSW,SWELL,7.1,227,2020-07-03 22:40:00+00:00
3,2020,07,03,21,40,0.7,0.7,19.0,0.2,4.2,SW,WSW,NaN,6.9,232,2020-07-03 21:40:00+00:00
4,2020,07,03,20,40,0.7,0.6,13.8,0.2,2.2,SW,W,NaN,6.6,219,2020-07-03 20:40:00+00:00
5,2020,07,03,19,40,0.8,0.7,19.0,0.2,4.3,WSW,W,NaN,8.2,237,2020-07-03 19:40:00+00:00


In [ ]:
station_id = "46025"

In [ ]:
#export
from huey.data.sql import get_db_engine

In [ ]:
db_engine = get_db_engine()

In [ ]:
def get_latest_ob(station_id):
    sql = f"select * from buoy_realtime_wave_detail order by ts desc limit 1"
    result = pd.read_sql(sql, db_engine)
    return result


In [ ]:
latest_ob = get_latest_ob(station_id).iloc[0]

In [ ]:
latest_ob.ts

Timestamp('2020-07-03 22:40:00+0000', tz='UTC')

In [ ]:
#export
def ts_from_df_row(row):
        return datetime(
            int(row['#YY']), 
            int(row['MM']), 
            int(row['DD']), 
            int(row['hh']), 
            int(row['mm']), 
            tzinfo=timezone.utc)

In [ ]:
row = df.iloc[1];
ts_from_df_row(row)

datetime.datetime(2020, 7, 3, 23, 40, tzinfo=datetime.timezone.utc)

## Final Function

In [61]:
#export

from huey.data.sql import get_db_engine

def import_buoy_realtime_wave_detail(station_id):
    db_engine = get_db_engine()
    sql = f"select * from buoy where station_id='{station_id}'"
    buoy = pd.read_sql(sql, db_engine).iloc[0]
    print(f"buoy: {buoy.station_id} id: {buoy.id}")
    
    sql = f"select * from buoy_realtime_wave_detail where buoy_id={buoy.id} order by ts desc limit 1"
    latest_ob = pd.read_sql(sql, db_engine, parse_dates=['ts'])
    print(f"latest_ob: {latest_ob}")
    print(f"len latest_ob {len(latest_ob)}")
    
    
    realtime_url = f"https://www.ndbc.noaa.gov/data/realtime2/{station_id}.spec"
    df = pd.read_csv(realtime_url, delim_whitespace=True)
    
    df["buoy_id"] = buoy.id

    df = df.replace('MM', np.NaN)

    df2 = df.rename(columns={
        "WVHT": "significant_wave_height",
        "SwH": "swell_height",
        "SwP": "swell_period",
        "SwD": "swell_direction",
        "WWH": "wind_wave_height",
        "WWP": "wind_wave_period",
        "WWD": "wind_wave_direction",
        "STEEPNESS": "steepness",
        "APD": "average_wave_period",
        "MWD": "dominant_wave_direction"
    })

    # skip first row
    df2 = df2[1:]

    def ts_from_df_row(row):
        return datetime(
            int(row['#YY']), 
            int(row['MM']), 
            int(row['DD']), 
            int(row['hh']), 
            int(row['mm']), 
            tzinfo=timezone.utc)

    df2["ts"] = df2.apply(lambda row: ts_from_df_row(row), axis=1)
    df2 = df2.drop(columns=['#YY', 'MM', 'DD', 'hh', 'mm'])
    #df2 = df2.drop(columns=['MWD'])
    
    if len(latest_ob) > 0:
        # only insert latest obs not in the db already
        df2_latest = df2[df2.ts > latest_ob.iloc[0].ts]
    else:
        df2_latest = df2
    
    df2_latest.to_sql('buoy_realtime_wave_detail', con=db_engine, if_exists='append', index=False)    

    return df2_latest

In [53]:
df_test = import_buoy_realtime_wave_detail("46011")

buoy: 46011 id: 429
latest_ob: Empty DataFrame
Columns: [id, buoy_id, ts, significant_wave_height, swell_height, swell_period, swell_direction, wind_wave_height, wind_wave_period, steepness, average_wave_period, dominant_wave_direction, created_at, updated_at, wind_wave_direction]
Index: []
len latest_ob 0


In [54]:
len(df_test)

1094

In [55]:
df_test.head()

,significant_wave_height,swell_height,swell_period,wind_wave_height,wind_wave_period,swell_direction,wind_wave_direction,steepness,average_wave_period,dominant_wave_direction,buoy_id,ts
1,2.9,NaN,NaN,NaN,NaN,NaN,NaN,STEEP,6.4,316,429,2020-07-06 15:40:00+00:00
2,2.7,NaN,NaN,NaN,NaN,NaN,NaN,AVERAGE,6.2,316,429,2020-07-06 14:40:00+00:00
3,2.8,NaN,NaN,NaN,NaN,NaN,NaN,AVERAGE,6.4,322,429,2020-07-06 13:40:00+00:00
4,3.2,NaN,NaN,NaN,NaN,NaN,NaN,STEEP,6.7,316,429,2020-07-06 12:40:00+00:00
5,2.9,NaN,NaN,NaN,NaN,NaN,NaN,STEEP,6.5,320,429,2020-07-06 11:40:00+00:00


In [62]:
df_test = import_buoy_realtime_wave_detail("46011")

buoy: 46011 id: 429
latest_ob:      id  buoy_id                        ts  significant_wave_height  \
0  9386      429 2020-07-06 15:40:00+00:00                      2.9   

  swell_height swell_period swell_direction wind_wave_height wind_wave_period  \
0         None         None            None             None             None   

  steepness  average_wave_period  dominant_wave_direction created_at  \
0     STEEP                  6.4                      316       None   

  updated_at wind_wave_direction  
0       None                None  
len latest_ob 1


In [63]:
len(df_test)

0

# Import Buoy Spectral Data

In [ ]:
station_id = "46025"

In [ ]:
#export
from huey.data.sql import get_db_session
from huey.data.models.buoy import Buoy, BuoyRawSpectralWaveData

In [ ]:
#export
def import_buoy_raw_spectral_wave_data(station_id):
    db_session = get_db_session()
    
    buoy = db_session.query(Buoy).filter(Buoy.station_id == station_id).first()
    latest_ob = db_session.query(BuoyRawSpectralWaveData).filter(BuoyRawSpectralWaveData.buoy_id == buoy.id ).order_by(BuoyRawSpectralWaveData.ts.desc()).first()
    
    raw_spec_url = f"https://www.ndbc.noaa.gov/data/realtime2/{station_id}.data_spec"
    response = requests.get(raw_spec_url)
    data = response.text
    
    for line in data.splitlines()[1:]:
    
        ob = BuoyRawSpectralWaveData.from_data_line(line)
        ob.buoy = buoy

        if (latest_ob is None or ob.ts > latest_ob.ts):
            print(f"inserting observation for date: {ob.ts}")
            db_session.add(ob)
        else:
            print(f"observation for date: {ob.ts} already present, skipping.")
            break

    db_session.commit()
    print("import complete")

    return data

In [ ]:
import_buoy_raw_spectral_wave_data(station_id)

[2020-01-22 13:17:06,326] {base_hook.py:84} INFO - Using connection to: id: huey_dev. Host: localhost, Port: None, Schema: huey_dev, Login: postgres, Password: XXXXXXXX, extra: None
inserting observation for date: 2020-01-22 21:00:00+00:00
inserting observation for date: 2020-01-22 20:00:00+00:00
inserting observation for date: 2020-01-22 19:00:00+00:00
inserting observation for date: 2020-01-22 18:00:00+00:00
inserting observation for date: 2020-01-22 17:00:00+00:00
inserting observation for date: 2020-01-22 16:00:00+00:00
inserting observation for date: 2020-01-22 15:00:00+00:00
inserting observation for date: 2020-01-22 14:00:00+00:00
inserting observation for date: 2020-01-22 13:00:00+00:00
inserting observation for date: 2020-01-22 12:00:00+00:00
inserting observation for date: 2020-01-22 11:00:00+00:00
inserting observation for date: 2020-01-22 10:00:00+00:00
inserting observation for date: 2020-01-22 09:00:00+00:00
inserting observation for date: 2020-01-22 08:00:00+00:00
insert

'#YY  MM DD hh mm Sep_Freq  < spec_1 (freq_1) spec_2 (freq_2) spec_3 (freq_3) ... >\n2020 01 22 21 00 9.999 0.000 (0.033) 0.000 (0.038) 0.000 (0.043) 0.000 (0.048) 0.213 (0.053) 0.912 (0.058) 3.040 (0.063) 2.888 (0.068) 1.338 (0.073) 1.672 (0.078) 1.976 (0.083) 2.006 (0.088) 1.520 (0.093) 1.094 (0.100) 0.669 (0.110) 0.395 (0.120) 0.274 (0.130) 0.152 (0.140) 0.091 (0.150) 0.091 (0.160) 0.334 (0.170) 0.334 (0.180) 0.334 (0.190) 0.486 (0.200) 0.152 (0.210) 0.182 (0.220) 0.243 (0.230) 0.213 (0.240) 0.091 (0.250) 0.091 (0.260) 0.061 (0.270) 0.091 (0.280) 0.061 (0.290) 0.061 (0.300) 0.030 (0.310) 0.030 (0.320) 0.000 (0.330) 0.000 (0.340) 0.030 (0.350) 0.000 (0.365) 0.000 (0.385) 0.000 (0.405) 0.000 (0.425) 0.000 (0.445) 0.000 (0.465) 0.000 (0.485) \n2020 01 22 20 00 0.250 0.000 (0.033) 0.000 (0.038) 0.000 (0.043) 0.000 (0.048) 0.449 (0.053) 1.527 (0.058) 2.649 (0.063) 4.490 (0.068) 4.266 (0.073) 2.963 (0.078) 2.021 (0.083) 0.763 (0.088) 0.943 (0.093) 0.988 (0.100) 0.629 (0.110) 0.180 (0.120)

# Export

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.sql.ipynb.
Converted 02_data.importer.buoy_data.ipynb.
Converted 03_dag.buoy_data.ipynb.
Converted 04_data.models.buoy.ipynb.
Converted 06_data.exporter.ipynb.
